In [1]:
! pip3 install numpy
! pip3 install pandas
! pip3 install torch

^C
ERROR: Operation cancelled by user


In [ ]:
! pip3 install nltk

In [ ]:
! pip3 install stanza

In [ ]:
! pip3 install scikit-learn

In [ ]:
! pip3 install pydantic==1.10.8

  Using cached pydantic-1.10.8-cp39-cp39-macosx_10_9_x86_64.whl.metadata (146 kB)
Using cached pydantic-1.10.8-cp39-cp39-macosx_10_9_x86_64.whl (2.9 MB)
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.8.2
    Uninstalling pydantic-1.8.2:
      Successfully uninstalled pydantic-1.8.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spacy 3.3.3 requires pydantic!=1.8,!=1.8.1,<1.9.0,>=1.7.4, but you have pydantic 1.10.8 which is incompatible.
thinc 8.0.17 requires pydantic!=1.8,!=1.8.1,<1.9.0,>=1.7.4, but you have pydantic 1.10.8 which is incompatible.


In [ ]:
! pip3 install allennlp

^C
ERROR: Operation cancelled by user


In [2]:
import numpy as np
print("GOT HERE 1")
import pandas as pd
print("GOT HERE 2")
import json
print("GOT HERE 3")
import nltk
print("GOT HERE 4")
from data_utils import BuildPOS
print("GOT HERE 6")
import torch.utils.data as data 
print("GOT HERE 7")
import time
print("GOT HERE 8")
import torch
print("GOT HERE 9")

GOT HERE 1


/var/folders/h1/ckvqd8ys0w18r_lfpbms82l80000gn/T/ipykernel_9887/3810321052.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


GOT HERE 2
GOT HERE 3
GOT HERE 4


/Users/tiberiuscolina/Desktop/ComputerStuff/School/nlpfinancial/finegrainedevents/src/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


GOT HERE 6
GOT HERE 7
GOT HERE 8
GOT HERE 9


In [4]:
from process_text import BuildVocab, TextToSentences

2024-02-15 20:48:02 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2024-02-15 20:48:03 INFO: Loading these models for language: en (English):
| Processor | Package         |
-------------------------------
| tokenize  | combined        |
| mwt       | combined        |
| pos       | combined_charlm |

2024-02-15 20:48:03 INFO: Using device: cpu
2024-02-15 20:48:03 INFO: Loading: tokenize
2024-02-15 20:48:03 INFO: Loading: mwt
2024-02-15 20:48:03 INFO: Loading: pos


At LSTM constructor
Started RNNBase constructor
Finished taking variables from params
Checked if dropout is between 0 and 1
Made through checks
Currently on layer 0
Completed dealing with layers
Parameters flattened
Parameters reset
LSTM constructor complete
At LSTM constructor
Started RNNBase constructor
Finished taking variables from params
Checked if dropout is between 0 and 1
Made through checks
Currently on layer 0
Completed dealing with layers
Parameters flattened
Parameters reset
LSTM constructor complete
At LSTM constructor
Started RNNBase constructor
Finished taking variables from params
Checked if dropout is between 0 and 1
Made through checks
Currently on layer 0
Completed dealing with layers
Parameters flattened
Parameters reset
LSTM constructor complete
At LSTM constructor
Started RNNBase constructor
Finished taking variables from params
Checked if dropout is between 0 and 1
Made through checks
Currently on layer 0
Completed dealing with layers
Parameters flattened
Paramet

2024-02-15 20:48:04 INFO: Done loading processors!
2024-02-15 20:48:04 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2024-02-15 20:48:05 INFO: Loading these models for language: en (English):
| Processor | Package           |
---------------------------------
| tokenize  | combined          |
| mwt       | combined          |
| pos       | combined_charlm   |
| lemma     | combined_nocharlm |
| depparse  | combined_charlm   |

2024-02-15 20:48:05 INFO: Using device: cpu
2024-02-15 20:48:05 INFO: Loading: tokenize
2024-02-15 20:48:05 INFO: Loading: mwt
2024-02-15 20:48:05 INFO: Loading: pos


At LSTM constructor
Started RNNBase constructor
Finished taking variables from params
Checked if dropout is between 0 and 1
Made through checks
Currently on layer 0
Completed dealing with layers
Parameters flattened
Parameters reset
LSTM constructor complete
At LSTM constructor
Started RNNBase constructor
Finished taking variables from params
Checked if dropout is between 0 and 1
Made through checks
Currently on layer 0
Completed dealing with layers
Parameters flattened
Parameters reset
LSTM constructor complete
At LSTM constructor
Started RNNBase constructor
Finished taking variables from params
Checked if dropout is between 0 and 1
Made through checks
Currently on layer 0
Completed dealing with layers
Parameters flattened
Parameters reset
LSTM constructor complete
At LSTM constructor
Started RNNBase constructor
Finished taking variables from params
Checked if dropout is between 0 and 1
Made through checks
Currently on layer 0
Completed dealing with layers
Parameters flattened
Paramet

2024-02-15 20:48:06 INFO: Loading: lemma
2024-02-15 20:48:06 INFO: Loading: depparse


Parameters reset
LSTM constructor complete
At LSTM constructor
Started RNNBase constructor
Finished taking variables from params
Checked if dropout is between 0 and 1
Made through checks
Currently on layer 0
Completed dealing with layers
Parameters flattened
Parameters reset
LSTM constructor complete
At LSTM constructor
Started RNNBase constructor
Finished taking variables from params
Checked if dropout is between 0 and 1
Made through checks
Currently on layer 0
Completed dealing with layers
Parameters flattened
Parameters reset
LSTM constructor complete
At LSTM constructor
Started RNNBase constructor
Finished taking variables from params
Checked if dropout is between 0 and 1
Made through checks
Currently on layer 0
Completed dealing with layers
Parameters flattened
Parameters reset
LSTM constructor complete
At LSTM constructor
Started RNNBase constructor
Finished taking variables from params
Checked if dropout is between 0 and 1
Made through checks
Currently on layer 0
Completed deali

2024-02-15 20:48:06 INFO: Done loading processors!


Parameters reset
LSTM constructor complete


In [5]:
def pretty_print(epoch, loss):
    print('Epoch {} completed with loss {:.4f}'.format(epoch + 1, loss * (0.9 ** epoch)))

In [6]:
with open('data.json') as f:
    dataset = json.load(f)

In [7]:
vocab = BuildVocab()
start = time.time()
for i in range(len(dataset)):
    text = TextToSentences(dataset[i]['news'].lower())
    vocab.addText(text)
print('Vocabulary built in {:.4f} sec, with size: {}'.format(time.time() - start, vocab.n_words))

Vocabulary built in 0.9079 sec, with size: 6326


In [8]:
pos = BuildPOS()
start = time.time()
for i in range(len(dataset)):
    p = dataset[i]['pos']
    pos.addPOS(p)
print('POS built in {:.4f} sec, with size: {}'.format(time.time() - start, pos.n_pos))

POS built in 0.0005 sec, with size: 16


In [9]:
class StockPredictionDataset(data.Dataset):
    def __init__(self, dataset, vocab, pos):
        self.data = dataset
        self.vocab = vocab
        self.pos = pos
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, i):
        news_, pos_, stock_ = self.data[i]['news'], self.data[i]['pos'], self.data[i]['stock']
        news_ = torch.Tensor(self.process_sent(news_)).long()
        pos_ = torch.Tensor(self.process_pos(news_, pos_)).long()
        stock_ = np.array(stock_)
        y = int(np.sum((stock_[-1,:6] - stock_[-2,:6])) > 0) if len(stock_) >= 2 else 1
        stock_ = torch.Tensor(stock_)
        return news_, pos_, stock_, y
    
    def process_sent(self, sent):
        tokens = nltk.word_tokenize(sent.lower())
        out = []
        out.append(self.vocab.word2index['<SOS>'])
        out.extend([self.vocab.word2index[token] for token in tokens if token in self.vocab.word2index])
        out.append(self.vocab.word2index['<EOS>'])
        return out
    
    def process_pos(self, sent, pos):
        out = [self.pos.pos2index['<SOS>']]
        out.extend([self.pos.pos2index[token] for token in pos])
        if len(sent) <= (len(pos) + 1):
            pos = pos[:len(sent) - 2] 
        out.extend([self.pos.pos2index[pos[-1]] for _ in range(len(sent) - len(pos) - 2) ])
        out.append(self.pos.pos2index['<EOS>'])
        return out
            

In [10]:
def collate_fn(data):
    def merge(sequences, is_token):
        lengths = [len(seq) for seq in sequences]
        if is_token:
            padded_seqs = torch.zeros(len(sequences), max(lengths)).long()
        else:
            padded_seqs = torch.zeros(len(sequences), max(lengths), 120)
        for i, seq in enumerate(sequences):
            end = lengths[i]
            if end == 0:
                continue
            padded_seqs[i,:end] = seq[:]
        return padded_seqs, lengths
              
    news, pos, stock, y = [], [], [], []
    for i, (news_i, pos_i, stock_i, y_i) in enumerate(data):
        news.append(news_i)
        pos.append(pos_i)
        stock.append(stock_i)
        y.append(y_i)
    news.sort(key = lambda x: len(x), reverse=True)
    pos.sort(key = lambda x: len(x), reverse=True)
    stock.sort(key = lambda x: len(x), reverse=True)
    news, news_lengths = merge(news, True)
    pos, _ = merge(pos, True)
    stock, stock_lengths = merge(stock, False)
    y = torch.Tensor(y)
    return news, pos, stock, y, news_lengths, stock_lengths
    

In [11]:
dataset = StockPredictionDataset(dataset, vocab, pos)
dataloader = torch.utils.data.DataLoader(dataset = dataset, batch_size=64, shuffle=True, collate_fn=collate_fn)

In [12]:
from model import MSSPM
import torch.nn as nn

In [ ]:
word_emb_dim = 256
hidden_dim = 256
vocab_size = vocab.n_words
event_size = pos.n_pos
num_heads = 4
num_epochs = 10
criterion = nn.BCELoss()

In [ ]:
# model = SSPM(vocab_size, word_emb_dim, hidden_dim, event_size, num_heads)
# model.cuda()
# optimizer = torch.optim.SGD(model.parameters(), lr=1e-2)

# for e in range(num_epochs):
#     for i, (news, pos, stock, y, sent_lengths, stock_lengths) in enumerate(dataloader):
#         bsize = len(news)
#         for i in range(bsize):
#             if sent_lengths[i] == 0:
#                 sent_lengths[i] += 1
#             elif stock_lengths[i] == 0:
#                 stock_lengths[i] += 1
#         optimizer.zero_grad()
        
#         """
#         news = news.cuda()
#         pos = pos.cuda()
#         stock = stock.cuda()
#         y = y.cuda().view(-1, 1)
#         probs = model(news, pos, stock, sent_lengths, stock_lengths)
#         loss = criterion(probs, y)
#         loss.backward()
#         optimizer.step()
#     pretty_print(e, loss.item())   
#     """   

In [13]:
word_emb_dim = 256
hidden_dim = 256
vocab_size = vocab.n_words
event_size = pos.n_pos
num_heads = 4
num_epochs = 10
eta = 0.5
criterion = nn.BCELoss()

pos.n_pos

(hidden_dim * 2) % num_heads == 0

True

In [14]:
model = MSSPM(vocab_size, word_emb_dim, hidden_dim, event_size, num_heads)
print("GOT HERE 12")
model.cuda()
print("GOT HERE 13")
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2)
print("GOT HERE 14")
for e in range(num_epochs):
    print(f"GOT HERE 15.{e}")
    for i, (news, pos, stock, y, sent_lengths, stock_lengths) in enumerate(dataloader):
        bsize = len(news)
        for i in range(bsize):
            if sent_lengths[i] == 0:
                sent_lengths[i] += 1
            elif stock_lengths[i] == 0:
                stock_lengths[i] += 1
        optimizer.zero_grad()
        news = news.cuda()
        pos = pos.cuda()
        stock = stock.cuda()
        y = y.cuda().view(-1, 1)
        probs, loss_crf = model(news, pos, stock, sent_lengths, stock_lengths)
        loss_bce = criterion(probs, y)
        loss = eta * loss_bce + (1 - eta) * loss_crf
        loss.backward()
        optimizer.step()
    pretty_print(e, loss.item())     

IS THE CODE GETTING HERE
IT SHOULD BE GETTING HERE TOO
GOT HERE 2
GOT HERE 3
Value of dropout is 4
Value of dropout is 0.2
At LSTM constructor
Started RNNBase constructor
Finished taking variables from params
Checked if dropout is between 0 and 1
Made through checks
Currently on layer 0
Currently on layer 1
Currently on layer 2
Currently on layer 3
Currently on layer 4
Currently on layer 5
Currently on layer 6
Currently on layer 7
Currently on layer 8
Currently on layer 9
Currently on layer 10
Currently on layer 11
Currently on layer 12
Currently on layer 13
Currently on layer 14
Currently on layer 15
Currently on layer 16
Currently on layer 17
Currently on layer 18
Currently on layer 19
Currently on layer 20
Currently on layer 21
Currently on layer 22
Currently on layer 23
Currently on layer 24
Currently on layer 25
Currently on layer 26
Currently on layer 27
Currently on layer 28
Currently on layer 29
Currently on layer 30
Currently on layer 31
Currently on layer 32
Currently on laye

: 

: 